Обучение Pipeline


In [1]:
import pandas as pd
import numpy as np
from my_pipeline_selectors import FeatureSelector, OHEEncoder, NumberSelector
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler

In [2]:
passengers_test = pd.read_csv('./test.csv')
passengers_train = pd.read_csv('./train.csv')
passengers_train_copy = passengers_train.copy()
passengers_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
passengers_train['Parch'].value_counts()

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

**сделать feature engineering**

In [ ]:
categorical_columns = ['Embarked', 'Sex']
continuous_columns = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

final_transformers = list()

scaler = StandardScaler()
scaler.fit(passengers_train[continuous_columns])

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
        ('selector', FeatureSelector(column=cat_col)),
        ('ohe', OHEEncoder(key=cat_col))
    ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
        ('selector', NumberSelector(key=cont_col)),
        ('standardizer', scaler)
    ])
    final_transformers.append((cont_col, cont_transformer))

In [5]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(final_transformers)
feature_processing = Pipeline([('feats', feats)])

In [6]:
from xgboost import XGBClassifier

xgboost_pipeline = Pipeline([
    ('features', feats),
    ('classifier', XGBClassifier(objective="binary:logistic", use_label_encoder=False, random_state=42)),
])

In [7]:
import dill

X = passengers_train.drop(columns=['Survived', 'Name', 'PassengerId', 'Ticket', 'Cabin'])
y = passengers_train['Survived']

xgboost_pipeline.fit(X, y)

with open("xgboost_pipeline.dill", "wb") as f:
    dill.dump(xgboost_pipeline, f)

[10:59:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [1]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import ShuffleSplit

# cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

# X = passengers_train.drop(columns=['Survived', 'Name', 'PassengerId', 'Ticket', 'Cabin'])
# y = passengers_train['Survived']

# xgboost_precision_scores = cross_val_score(xgboost_pipeline, X, y, cv=cv, scoring="precision_macro").mean()
# xgboost_recall_scores = cross_val_score(xgboost_pipeline, X, y, cv=cv, scoring="recall_macro").mean()
# xgboost_roc_auc_scores = cross_val_score(xgboost_pipeline, X, y, cv=cv, scoring="roc_auc").mean()
# xgboost_f1_score_scores = cross_val_score(xgboost_pipeline, X, y, cv=cv, scoring="f1_macro").mean()

In [2]:
# data = {
#     'xgboost': [
#         np.mean(xgboost_precision_scores),
#         np.mean(xgboost_recall_scores),
#         np.mean(xgboost_roc_auc_scores),
#         np.mean(xgboost_f1_score_scores)
#     ]
# }
# before = pd.DataFrame.from_dict(data, orient='index', columns=['precision', 'recall', 'roc_auc', 'f1_score'])
# before